<a href="https://colab.research.google.com/github/tallywiesenberg/DS-Unit-2-Applied-Modeling/blob/master/Unit_2_Build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install category_encoders

import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#pipeline imports
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegressionCV

#Import

In [11]:
#crash data
crashes = pd.read_csv('https://opendata.arcgis.com/datasets/70392a096a8e431381f1f692aaa06afd_24.csv', dtype={'STREETSEGID': 'Int64'})  #crash data
crashes1 = pd.read_csv('https://opendata.arcgis.com/datasets/70248b73c20f46b0a5ee895fc91d6222_25.csv', dtype={'STREETSEGID': 'Int64'}) #additional crash data
crashes.shape, crashes1.shape

IncompleteRead: ignored

In [0]:
crashes.head(2)

In [0]:
crashes1.head()

##Pre-merge cleaning

In [0]:
car_types = crashes1.groupby('CRIMEID')['INVEHICLETYPE'].value_counts().unstack().replace(np.nan, 0).reset_index()

In [0]:
crashes1['SENIORS'] = crashes1['AGE'] > 65
crashes1['CHILDREN'] = crashes1['AGE'] < 18

In [0]:
crashes1.groupby('CRIMEID')['SENIORS'].value_counts()

##Merge

In [0]:
#datasets for engineering
street_segs = pd.read_csv('https://opendata.arcgis.com/datasets/f21813094ed24fd5ba66a07b61a1acee_46.csv', dtype={'STREETSEGID': 'Int64'})
bicycles = pd.read_csv('https://opendata.arcgis.com/datasets/294e062cdf2c48d5b9cbc374d9709bc0_2.csv', dtype={'STREETSEGID': 'Int64'})

need to aggregate crashes1 (personal crash data)

In [0]:
df = crashes.merge(car_types, how='inner', on='CRIMEID')   #inner join on CRIMEID, the common unique ID between datasets
df = df.merge(street_segs[['STREETSEGID', 'STREETTYPE', 'QUADRANT', 'DIRECTIONALITY']], how='left', on='STREETSEGID')
df = df.merge(bicycles[['STREETSEGID', 'BIKELANELENGTH']], how='left', on='STREETSEGID')
df.shape

(216075, 82)

In [0]:
df= df.sort_index(axis=1)
df.head()

,ADDRESS,Aircraft,Atv (all Terrain Vehicle),BICYCLISTSIMPAIRED,BIKELANELENGTH,BLOCKKEY,Bus,CCN,CRIMEID,Cargo Van,Construction/industrial Equipment,DIRECTIONALITY,DRIVERSIMPAIRED,Drugs/ Narcotics,EVENTID,FATAL_BICYCLIST,FATAL_DRIVER,FATAL_PEDESTRIAN,FROMDATE,Farm Equipment,Firearms,INTAPPROACHDIRECTION,LASTUPDATEDATE,LATITUDE,LOCATIONERROR,LONGITUDE,Large/heavy Truck,MAJORINJURIES_BICYCLIST,MAJORINJURIES_DRIVER,MAJORINJURIES_PEDESTRIAN,MARID,MAR_ADDRESS,MAR_SCORE,MEASURE,MINORINJURIES_BICYCLIST,MINORINJURIES_DRIVER,MINORINJURIES_PEDESTRIAN,MPDGEOX,MPDGEOY,MPDLATITUDE,...,Motor Cycle,Motorhome/camper/rv (recreational Vehicle),NEARESTINTROUTEID,NEARESTINTSTREETNAME,None,OBJECTID,OFFINTERSECTION,OFFSET,Other Small/light Truck,Other Vehicle,PEDESTRIANSIMPAIRED,Passenger Car/automobile,Passenger Van,Pickup Truck,QUADRANT,REPORTDATE,ROADWAYSEGID,ROUTEID,SPEEDING_INVOLVED,STREETSEGID,STREETTYPE,SUBBLOCKKEY,Snow Mobile,Suv (sport Utility Vehicle),TODATE,TOTAL_BICYCLES,TOTAL_GOVERNMENT,TOTAL_PEDESTRIANS,TOTAL_TAXIS,TOTAL_VEHICLES,Trailer,UNKNOWNINJURIES_BICYCLIST,UNKNOWNINJURIES_DRIVER,UNKNOWNINJURIES_PEDESTRIAN,WARD,Watercraft/boat,X,XCOORD,Y,YCOORD
0,1633 NORTH CAPITOL STREET NE,0.0,0.0,0,NaN,NaN,0.0,18138880,27773808,0.0,0.0,2.0,0,0.0,{2717C7D8-1BD7-4550-A9A4-3D0B8CA06A95},0,0,0,2018-08-20T04:00:00.000Z,0.0,0.0,West,NaN,38.912419,NaN,-77.008584,0.0,0,0,0,58903,1633 NORTH CAPITOL STREET NE,200.0,151.349,0,1,0,399240.51,138287.26,38.912573,...,0.0,0.0,12074162,R ST NE,0.0,109265316,37.525917,15.342407,0.0,0.0,0,3.0,0.0,0.0,NE,2018-08-20T22:38:20.000Z,15095.0,12055642,0,10166,RD,NaN,0.0,0.0,NaN,0,0,0,0,3,0.0,0,0,0,Ward 5,0.0,-77.008587,399255.487,38.912427,138283.932
1,2455 ALABAMA AVENUE SE,0.0,0.0,0,NaN,NaN,0.0,18138896,27773815,0.0,0.0,2.0,0,0.0,{298310E1-A374-4515-BEEB-66467A0FBC85},0,0,0,2018-08-20T04:00:00.000Z,0.0,0.0,West,NaN,38.853037,NaN,-76.969521,0.0,0,0,0,278162,2455 ALABAMA AVENUE SE,200.0,39.500,0,1,0,402645.56,131727.88,38.853033,...,0.0,0.0,13009362,ALABAMA AVE SE,0.0,109265317,4.177299,35.458947,0.0,0.0,0,1.0,0.0,0.0,SE,2018-08-20T22:46:30.000Z,8260.0,13049502,0,7473,ST,NaN,0.0,0.0,NaN,0,0,0,0,2,0.0,0,0,0,Ward 8,0.0,-76.969523,402645.610,38.853044,131692.421
2,3905 DIX STREET NE,0.0,0.0,0,NaN,NaN,0.0,18138856,27773821,0.0,0.0,NaN,0,0.0,{31C4BA97-057F-46D8-B15E-CBA9A08108B1},0,0,0,2018-08-20T04:00:00.000Z,0.0,0.0,West,NaN,38.893255,NaN,-76.949491,0.0,0,1,0,29514,3905 DIX STREET NE,200.0,19.297,0,0,0,404387.75,136169.49,38.893530,...,1.0,0.0,12023662,CLAY PL NE,0.0,109265318,69.620746,13.202794,0.0,0.0,0,1.0,0.0,0.0,NaN,2018-08-20T23:26:35.000Z,24002.0,47015862,1,-9,NaN,NaN,0.0,0.0,NaN,0,0,0,0,2,0.0,0,0,0,Ward 7,0.0,-76.949494,404381.720,38.893263,136157.745
3,CONSTITUTION AVENUE NW & 10TH STREET NW,0.0,0.0,0,NaN,NaN,0.0,18138883,27773824,0.0,0.0,2.0,0,0.0,{F7CCD625-CAA6-4706-B97B-64B83E36EED3},0,0,0,2018-08-20T04:00:00.000Z,0.0,0.0,West,NaN,38.892082,NaN,-77.025990,0.0,0,0,0,293188,NaN,200.0,0.000,0,0,0,397745.24,136026.65,38.891962,...,0.0,0.0,11025352,CONSTITUTION AVE NW,0.0,109265319,88.932111,0.004743,0.0,0.0,0,2.0,0.0,0.0,NW,2018-08-20T23:01:56.000Z,6547.0,11001002,0,7822,AVE,NaN,0.0,0.0,NaN,0,0,0,1,2,0.0,0,0,0,Ward 2,0.0,-77.025993,397745.238,38.892090,136026.655
4,1700 1ST STREET NW,0.0,0.0,0,NaN,NaN,0.0,18138749,27773833,1.0,0.0,NaN,0,0.0,{77D12E4A-7CE6-468F-9FB8-554878921C0A},0,0,0,2018-08-20T04:00:00.000Z,0.0,0.0,West,NaN,38.912739,NaN,-77.012552,0.0,0,0,0,218404,1700 1ST STREET NW,200.0,28.773,0,0,0,398927.62,138321.30,38.913455,...,0.0,0.0,11074162,R ST NW,0.0,109265320,98.512188,16.323611,0.0,0.0,0,0.0,1.0,0.0,NaN,2018-08-20T23:20:38.000Z,29617.0,47070502,0,-9,NaN,NaN,0.0,0.0,NaN,0,1,0,0,2,0.0,0,0,0,Ward 5,0.0,-77.012554,398911.404,38.912746,138319.430


#Cleaning

In [0]:
df.head()

,ADDRESS,Aircraft,Atv (all Terrain Vehicle),BICYCLISTSIMPAIRED,BIKELANELENGTH,BLOCKKEY,Bus,CCN,CRIMEID,Cargo Van,Construction/industrial Equipment,DIRECTIONALITY,DRIVERSIMPAIRED,Drugs/ Narcotics,EVENTID,FATAL_BICYCLIST,FATAL_DRIVER,FATAL_PEDESTRIAN,FROMDATE,Farm Equipment,Firearms,INTAPPROACHDIRECTION,LASTUPDATEDATE,LATITUDE,LOCATIONERROR,LONGITUDE,Large/heavy Truck,MAJORINJURIES_BICYCLIST,MAJORINJURIES_DRIVER,MAJORINJURIES_PEDESTRIAN,MARID,MAR_ADDRESS,MAR_SCORE,MEASURE,MINORINJURIES_BICYCLIST,MINORINJURIES_DRIVER,MINORINJURIES_PEDESTRIAN,MPDGEOX,MPDGEOY,MPDLATITUDE,...,Motor Cycle,Motorhome/camper/rv (recreational Vehicle),NEARESTINTROUTEID,NEARESTINTSTREETNAME,None,OBJECTID,OFFINTERSECTION,OFFSET,Other Small/light Truck,Other Vehicle,PEDESTRIANSIMPAIRED,Passenger Car/automobile,Passenger Van,Pickup Truck,QUADRANT,REPORTDATE,ROADWAYSEGID,ROUTEID,SPEEDING_INVOLVED,STREETSEGID,STREETTYPE,SUBBLOCKKEY,Snow Mobile,Suv (sport Utility Vehicle),TODATE,TOTAL_BICYCLES,TOTAL_GOVERNMENT,TOTAL_PEDESTRIANS,TOTAL_TAXIS,TOTAL_VEHICLES,Trailer,UNKNOWNINJURIES_BICYCLIST,UNKNOWNINJURIES_DRIVER,UNKNOWNINJURIES_PEDESTRIAN,WARD,Watercraft/boat,X,XCOORD,Y,YCOORD
0,1633 NORTH CAPITOL STREET NE,0.0,0.0,0,NaN,NaN,0.0,18138880,27773808,0.0,0.0,2.0,0,0.0,{2717C7D8-1BD7-4550-A9A4-3D0B8CA06A95},0,0,0,2018-08-20T04:00:00.000Z,0.0,0.0,West,NaN,38.912419,NaN,-77.008584,0.0,0,0,0,58903,1633 NORTH CAPITOL STREET NE,200.0,151.349,0,1,0,399240.51,138287.26,38.912573,...,0.0,0.0,12074162,R ST NE,0.0,109265316,37.525917,15.342407,0.0,0.0,0,3.0,0.0,0.0,NE,2018-08-20T22:38:20.000Z,15095.0,12055642,0,10166,RD,NaN,0.0,0.0,NaN,0,0,0,0,3,0.0,0,0,0,Ward 5,0.0,-77.008587,399255.487,38.912427,138283.932
1,2455 ALABAMA AVENUE SE,0.0,0.0,0,NaN,NaN,0.0,18138896,27773815,0.0,0.0,2.0,0,0.0,{298310E1-A374-4515-BEEB-66467A0FBC85},0,0,0,2018-08-20T04:00:00.000Z,0.0,0.0,West,NaN,38.853037,NaN,-76.969521,0.0,0,0,0,278162,2455 ALABAMA AVENUE SE,200.0,39.500,0,1,0,402645.56,131727.88,38.853033,...,0.0,0.0,13009362,ALABAMA AVE SE,0.0,109265317,4.177299,35.458947,0.0,0.0,0,1.0,0.0,0.0,SE,2018-08-20T22:46:30.000Z,8260.0,13049502,0,7473,ST,NaN,0.0,0.0,NaN,0,0,0,0,2,0.0,0,0,0,Ward 8,0.0,-76.969523,402645.610,38.853044,131692.421
2,3905 DIX STREET NE,0.0,0.0,0,NaN,NaN,0.0,18138856,27773821,0.0,0.0,NaN,0,0.0,{31C4BA97-057F-46D8-B15E-CBA9A08108B1},0,0,0,2018-08-20T04:00:00.000Z,0.0,0.0,West,NaN,38.893255,NaN,-76.949491,0.0,0,1,0,29514,3905 DIX STREET NE,200.0,19.297,0,0,0,404387.75,136169.49,38.893530,...,1.0,0.0,12023662,CLAY PL NE,0.0,109265318,69.620746,13.202794,0.0,0.0,0,1.0,0.0,0.0,NaN,2018-08-20T23:26:35.000Z,24002.0,47015862,1,-9,NaN,NaN,0.0,0.0,NaN,0,0,0,0,2,0.0,0,0,0,Ward 7,0.0,-76.949494,404381.720,38.893263,136157.745
3,CONSTITUTION AVENUE NW & 10TH STREET NW,0.0,0.0,0,NaN,NaN,0.0,18138883,27773824,0.0,0.0,2.0,0,0.0,{F7CCD625-CAA6-4706-B97B-64B83E36EED3},0,0,0,2018-08-20T04:00:00.000Z,0.0,0.0,West,NaN,38.892082,NaN,-77.025990,0.0,0,0,0,293188,NaN,200.0,0.000,0,0,0,397745.24,136026.65,38.891962,...,0.0,0.0,11025352,CONSTITUTION AVE NW,0.0,109265319,88.932111,0.004743,0.0,0.0,0,2.0,0.0,0.0,NW,2018-08-20T23:01:56.000Z,6547.0,11001002,0,7822,AVE,NaN,0.0,0.0,NaN,0,0,0,1,2,0.0,0,0,0,Ward 2,0.0,-77.025993,397745.238,38.892090,136026.655
4,1700 1ST STREET NW,0.0,0.0,0,NaN,NaN,0.0,18138749,27773833,1.0,0.0,NaN,0,0.0,{77D12E4A-7CE6-468F-9FB8-554878921C0A},0,0,0,2018-08-20T04:00:00.000Z,0.0,0.0,West,NaN,38.912739,NaN,-77.012552,0.0,0,0,0,218404,1700 1ST STREET NW,200.0,28.773,0,0,0,398927.62,138321.30,38.913455,...,0.0,0.0,11074162,R ST NW,0.0,109265320,98.512188,16.323611,0.0,0.0,0,0.0,1.0,0.0,NaN,2018-08-20T23:20:38.000Z,29617.0,47070502,0,-9,NaN,NaN,0.0,0.0,NaN,0,1,0,0,2,0.0,0,0,0,Ward 5,0.0,-77.012554,398911.404,38.912746,138319.430


In [0]:
#create time series from date of crash
df['FROMDATE'] = pd.to_datetime(df['FROMDATE'].str.slice(stop=10), infer_datetime_format = True)
df['FROMDATE'].head()

0   2018-08-20
1   2018-08-20
2   2018-08-20
3   2018-08-20
4   2018-08-20
Name: FROMDATE, dtype: datetime64[ns]

In [0]:
#drop observations recorded before 2000 because they are date entry errors (judgement call)

df.drop(df.query('FROMDATE < 2000').index, inplace=True)


In [0]:
df['FROMDATE'].describe()

count                  216027
unique                   4361
top       2019-06-13 00:00:00
freq                      251
first     2000-01-01 00:00:00
last      2019-10-03 00:00:00
Name: FROMDATE, dtype: object

In [0]:
#drop duplicate and ID columns
df = df.drop(columns=['OBJECTID', 'CCN', 'EVENTID', 'BLOCKKEY', 'EVENTID', 'None'])  #"none" column is leftover from car_types merge, it means no cars were involved (illogical)

In [0]:
#cleaning STREET SEGMENT ID
df['STREETSEGID'] = (df['STREETSEGID'].replace(-9.0, np.nan)   #-9 is not a valid street segment id
                                      .replace(0.0, np.nan)    #0 isnt a valid street segment id
#                                       .astype(int)
                    )
#cleaning WARD attribute
mode = df['WARD'].mode()[0]                                      #mode of WARD attribute

df['WARD'] = df['WARD'].replace('Null', np.nan).fillna(value=df['WARD'].mode()[0])  #fill WARD attribute with mode of column (useful so column can be converted to int dtype)
df['WARD'] = df['WARD'].str.replace('Ward ', '').astype(int)   #remove "ward" string from ward attribute


In [0]:
# impute_mode = SimpleImputer(strategy='most_frequent')
# impute_mean = SimpleImputer(strategy='median')

# impute_mode.fit_transform(df.select_dtypes('object'))
# impute_mean.fit_transform(df.select_dtypes('number'))

In [0]:
#change Y/N to 0/1
#will return an error, but still works?
diction = {'N': 0, 'Y': 1}
for col in df.columns:
  if (df[col].unique().tolist() == ['N', 'Y']) | (df[col].unique().tolist() == ['Y', 'N']):
    df[col] = df[col].replace(diction)
  else:
    pass

In [0]:
#change missing values of BIKELANELENGTH attribute to 0s
#soon to be dropped, just need to know if there is a bikelane or not
df['BIKELANELENGTH'] = df['BIKELANELENGTH'].replace(np.nan, 0)

In [0]:
#drop columns where missing values are 35 percent or more of column

for col in df.columns:
  if df[col].isna().sum() >= .5 * df.shape[0]:
    df = df.drop(columns=col)
  else:
    pass
  
df.shape

#Engineering

In [0]:
#all the features that will be summed to become the target variable

target_cols = df.columns[df.columns.str.contains('FATAL') | df.columns.str.contains('MAJOR')].tolist()

target_numeric = df[target_cols].agg('sum', axis='columns')  #target column formatted numerically

target_mask = target_numeric > 0                             #target mask returns with either True (major injury or fatality) or False (no major injury or fatality)

df['MAJOR_INJURY_OR_FATAL'] = target_mask

diction = {False: 0, True: 1}                                #change True/False to 0/1
df['MAJOR_INJURY_OR_FATAL'] = df['MAJOR_INJURY_OR_FATAL'].replace(diction)

In [0]:
df['MAJOR_INJURY_OR_FATAL'].value_counts()


In [0]:
##Bike lanes
diction = {False: 1, True: 0}

df['BIKE_LANE'] = df['BIKELANELENGTH'] == 0        #is there a bikelane (this is after NaNs are replaced with 0s)
df['BIKE_LANE'] = df['BIKE_LANE'].replace(diction) #0 means no bikelane, 1 means bikelane

df['BIKE_LANE'].value_counts()

#drop BIKELANELENGTH, YEARINSTALLED, no longer useful
print(df.shape)
df = df.drop('BIKELANELENGTH', axis = 1)
print(df.shape)

In [0]:
#YEAR of accident

df['YEAR'] = df['FROMDATE'].dt.year
df['YEAR'].value_counts()

#Baseline Model

In [0]:
df['MAJOR_INJURY_OR_FATAL'].value_counts()

In [0]:
df['MAJOR_INJURY_OR_FATAL'].value_counts(normalize=True)

In [0]:
y_pred = [0] * df.shape[0]
print('Baseline accuracy score:', accuracy_score(df['MAJOR_INJURY_OR_FATAL'], y_pred))
#Despite high accuracy, baseline model is useless because false negatives are very costly to society
print(confusion_matrix(df['MAJOR_INJURY_OR_FATAL'], y_pred))

#Exploration

In [0]:
df.head(2)

In [0]:
df.groupby('WARD')['MAJOR_INJURY_OR_FATAL'].mean().plot.bar()
plt.title('Probability of Major Injury or Death from Car Accident by DC Ward')

In [0]:
df.groupby('BIKE_LANE')['MAJOR_INJURY_OR_FATAL'].mean().plot.bar()

In [0]:
df.groupby('DRIVERSIMPAIRED')['MAJOR_INJURY_OR_FATAL'].mean().plot.bar()

In [0]:
df.groupby('SPEEDING')['MAJOR_INJURY_OR_FATAL'].mean().plot.bar()

In [0]:
df.groupby('INTAPPROACHDIRECTION')['MAJOR_INJURY_OR_FATAL'].mean().plot.bar()

In [0]:
df.groupby('SPEEDING_INVOLVED')['MAJOR_INJURY_OR_FATAL'].mean().plot.bar()

In [0]:
df.groupby('STREETTYPE')['MAJOR_INJURY_OR_FATAL'].mean().plot.bar()

In [0]:
df['OFFINTERSECTION'].hist(range=(0, 250), bins=25)

In [0]:
px.set_mapbox_access_token(open(".mapbox_token").read())

px.scatter_mapbox(df, lat='X', lon= 'Y', color='MAJOR_INJURY_OR_FATAL')

#Split

In [0]:
test_mask = df['YEAR'] >= 2019
train = df[~test_mask]
test = df[test_mask]

val_mask = train['YEAR'] < 2017
val = train[~val_mask]
train = train[val_mask]


train.shape, val.shape, test.shape

In [0]:
target = 'MAJOR_INJURY_OR_FATAL'
features = df.columns[df.columns != target]

X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]
y_test = test[target]

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

#Logistic Regression

In [0]:
pipeline = make_pipeline(
    OneHotEncoder(),
    SimpleImputer(strategy='median'),
    LogisticRegressionCV())

In [0]:
pipeline.fit(X_train, y_train)

pipeline.score(X_val, y_val)